<a href="https://colab.research.google.com/github/lokeshmandarapu/Chatbot/blob/main/automated_code_generator.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Install the required library
!pip install google-cloud-aiplatform --upgrade --user

# Restart the kernel after installing the package (specific to Colab)
import IPython
IPython.Application.instance().kernel.do_shutdown(True)

# Authenticate your notebook environment (for Colab users)
from google.colab import auth
auth.authenticate_user()

# Import necessary libraries
from vertexai.language_models import CodeGenerationModel
import vertexai
import os

# Initialize Vertex AI
PROJECT_ID = "your-project-id"  # Replace with your Google Cloud Project ID
vertexai.init(project=PROJECT_ID, location="us-central1")

# Load the code generation model
code_generation_model = CodeGenerationModel.from_pretrained("code-bison@002")

# History of generated code
generated_code_history = []

def generate_code(prompt, max_tokens=512, temperature=0.5):
    """
    Generate code based on a user prompt.

    Args:
        prompt (str): The natural language description for the code.
        max_tokens (int): Maximum output tokens for the code.
        temperature (float): Controls randomness in code generation (0.0-1.0).

    Returns:
        str: The generated code.
    """
    response = code_generation_model.predict(
        prefix=prompt,
        max_output_tokens=max_tokens,
        temperature=temperature,
    )
    return response.text

def save_code_to_file(file_name, code):
    """
    Save the generated code to a file.

    Args:
        file_name (str): The name of the file to save the code.
        code (str): The code to save.
    """
    with open(file_name, "w") as file:
        file.write(code)
    print(f"Code saved to {file_name}")

def choose_language():
    """
    Display a menu for choosing a programming language.

    Returns:
        str: The selected programming language.
    """
    languages = [
        "Python", "JavaScript", "Java", "C++", "Go", "Rust", "PHP",
        "TypeScript", "Kotlin", "Swift", "Scala", "GoogleSQL"
    ]
    print("\nChoose a programming language:")
    for idx, lang in enumerate(languages, start=1):
        print(f"{idx}. {lang}")
    choice = int(input("Enter the number corresponding to your choice: "))
    return languages[choice - 1]

def show_example_prompts():
    """
    Display example prompts to guide the user.
    """
    print("\nExample Prompts:")
    examples = [
        "Write a Python function to calculate the factorial of a number.",
        "Generate a SQL query to find the total sales by product category.",
        "Write a JavaScript function to filter an array of objects by a property value.",
        "Write a C++ program to reverse a string."
    ]
    for example in examples:
        print(f"- {example}")

# Main interactive loop
while True:
    print("\n--- Code Generator ---")
    print("1. Generate code")
    print("2. Generate code with a specific programming language")
    print("3. View generated code history")
    print("4. Clear generated code history")
    print("5. Show example prompts")
    print("6. Exit")
    option = input("Choose an option (1/2/3/4/5/6): ").strip()

    if option == "1":
        user_prompt = input("\nEnter your prompt to generate code: ").strip()
        max_tokens = int(input("Enter maximum output tokens (default 512): ") or "512")
        temperature = float(input("Enter temperature (0.0 to 1.0, default 0.5): ") or "0.5")
        generated_code = generate_code(user_prompt, max_tokens, temperature)
        print("\nGenerated Code:\n")
        print(generated_code)

        # Save the generated code in history
        generated_code_history.append({"prompt": user_prompt, "code": generated_code})

        save_option = input("\nDo you want to save this code to a file? (yes/no): ").strip().lower()
        if save_option == "yes":
            file_name = input("Enter the file name (e.g., my_code.py): ").strip()
            save_code_to_file(file_name, generated_code)

    elif option == "2":
        language = choose_language()
        user_prompt = input(f"\nEnter your prompt to generate {language} code: ").strip()
        max_tokens = int(input("Enter maximum output tokens (default 512): ") or "512")
        temperature = float(input("Enter temperature (0.0 to 1.0, default 0.5): ") or "0.5")
        generated_code = generate_code(f"Write {language} code: {user_prompt}", max_tokens, temperature)
        print("\nGenerated Code:\n")
        print(generated_code)

        # Save the generated code in history
        generated_code_history.append({"prompt": user_prompt, "code": generated_code})

        save_option = input("\nDo you want to save this code to a file? (yes/no): ").strip().lower()
        if save_option == "yes":
            file_extension = {
                "Python": ".py", "JavaScript": ".js", "Java": ".java", "C++": ".cpp",
                "Go": ".go", "Rust": ".rs", "PHP": ".php", "TypeScript": ".ts",
                "Kotlin": ".kt", "Swift": ".swift", "Scala": ".scala", "GoogleSQL": ".sql"
            }
            file_name = f"generated_code{file_extension.get(language, '.txt')}"
            save_code_to_file(file_name, generated_code)

    elif option == "3":
        print("\n--- Generated Code History ---")
        if not generated_code_history:
            print("No generated code yet.")
        else:
            for idx, entry in enumerate(generated_code_history, start=1):
                print(f"{idx}. Prompt: {entry['prompt']}\nCode:\n{entry['code']}\n")

    elif option == "4":
        generated_code_history.clear()
        print("Generated code history cleared.")

    elif option == "5":
        show_example_prompts()

    elif option == "6":
        print("Exiting the code generator. Goodbye!")
        break

    else:
        print("Invalid option. Please try again.")
